<a href="https://www.kaggle.com/code/girishkalyan/bp-prediction?scriptVersionId=221720617" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

In [11]:
warnings.filterwarnings("ignore")


In [12]:
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")
train_data = pd.concat([train, train_extra], axis=0, ignore_index=True)
train_data = train_data.dropna()
test_data1 = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")
test_data = test_data1.dropna()


****Feature Extraction****

In [13]:
display(test_data1)


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
1,300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
2,300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
3,300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
4,300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953
...,...,...,...,...,...,...,...,...,...,...
199995,499995,Adidas,Canvas,Large,2.0,Yes,No,Messenger,Red,7.383498
199996,499996,Nike,Polyester,Small,9.0,No,Yes,Messenger,Pink,6.058394
199997,499997,Jansport,Nylon,Small,9.0,No,Yes,Tote,Green,26.890163
199998,499998,Puma,Nylon,Large,10.0,Yes,No,Tote,Gray,25.769153


In [14]:
# Encode categorical features
categorical_features = ['Brand', 'Material', 'Size', 'Style', 'Color']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(train_data[categorical_features])
categorical_data = pd.DataFrame(categorical_encoded, index=train_data.index, columns=encoder.get_feature_names_out(categorical_features))


In [15]:
# Encode categorical features for test data
categorical_encoded_test = encoder.fit_transform(test_data[categorical_features])
categorical_data_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out(categorical_features))

In [16]:
numerical_features = ['id','Compartments', 'Weight Capacity (kg)']
binary_features = ['Laptop Compartment', 'Waterproof']


In [17]:
train_data[binary_features] = train_data[binary_features].replace({'Yes': 1, 'No': 0})
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [18]:
X_train = pd.concat([train_data[numerical_features + binary_features], categorical_data], axis=1)
Y_train = train_data[["Price"]]
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [19]:
X_train.drop('id', axis =1
            )

,Compartments,Weight Capacity (kg),Laptop Compartment,Waterproof,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,...,Size_Small,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,7.0,11.611723,1,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10.0,27.078537,1,1,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,16.643760,1,0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8.0,12.937220,1,0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,17.749338,1,1,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994311,4.0,28.425845,1,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3994312,6.0,16.641739,1,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3994314,10.0,17.379531,1,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3994315,10.0,17.037708,0,0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


****Feature Scaling****

In [20]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [21]:
# Ensure your data is in NumPy array format
X_train = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train
Y_train = Y_train.to_numpy() if hasattr(Y_train, 'to_numpy') else Y_train
X_test = X_test.to_numpy() if hasattr(X_test, 'to_numpy') else X_test


**Random Forrest Initialization**

In [22]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [23]:
import tensorflow as tf

# Initialize the MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")


Number of devices: 2


In [24]:
with strategy.scope():
    model = RandomForestRegressor(
            n_estimators=100,            # Number of trees in the forest
            max_depth=None,              # Maximum depth of the tree (None allows full depth)
            min_samples_split=2,         # Minimum samples required to split an internal node
            min_samples_leaf=1,          # Minimum samples required to be a leaf node
            max_features='sqrt',         # Number of features to consider when looking for the best split
            bootstrap=True,              # Whether bootstrap samples are used when building trees
            random_state=42             # Random state for reproducibility
        
        )

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold


# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
scores = []

for train_index, val_index in kf.split(X_train):
    # Split the data
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    # Train the model
    model.fit(X_train_fold, Y_train_fold)

    # Evaluate the model
    score = model.score(X_val_fold, Y_val_fold)
    print(f"Fold {fold_no} - R2 Score: {score:.4f}")
    scores.append(score)

    fold_no += 1

# Average score across folds
print(f"Average R2 Score: {sum(scores) / len(scores):.4f}")


In [ ]:
model.predict(X_test)